LangchainのGraphRAGの記事  
https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/


In [ ]:
import os
from langchain_openai import (
    AzureOpenAIEmbeddings,
    OpenAIEmbeddings,
    AzureChatOpenAI,
    ChatOpenAI
)

from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv('../.env')

#### 1. Model読み込み

In [2]:
# emmbeddingsのモデルを取得
embeddings = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="embedding",
        openai_api_version="2024-06-01"
    )
elif os.getenv('OPENAI_API_KEY') != "":
    # OpenAIの場合
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
else:
    print("APIKeyの設定を確認してください")

# chatのモデルを取得
model = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    model = AzureChatOpenAI(
        azure_deployment="chat",
        openai_api_version="2024-06-01",
        temperature=0.
    )
elif os.getenv('OPENAI_API_KEY') != "":
    # OpenAIの場合
    model = ChatOpenAI(model="gpt-4")
else:
    print("APIKeyの設定を確認してください")

#### 2. データの準備

2-1. データ読み込み

In [3]:
import pandas as pd
df = pd.read_csv('data/工事マスター.csv', encoding="cp932")
df[["工事名", "備考"]].to_csv('data/temp.csv', encoding="cp932", index=False)

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(
    file_path="./data/temp.csv",
    encoding="cp932",
)
docs = loader.load()

In [ ]:
docs[:3]

2-2. Graph作成

In [6]:
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [7]:
llm_transformer = LLMGraphTransformer(llm=model)
# Extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(docs)

In [ ]:
print("graph_documents len",len(graph_documents)) 
print("graph_documents", graph_documents[:1])

In [9]:
# Create a graph
graph = Neo4jGraph()

# 実行するとデータの削除可能
# graph.query("MATCH (n) DETACH DELETE n")

# Store graph data
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

### 3. Unstructured data retriever


In [ ]:
from langchain_community.vectorstores import Neo4jVector
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid", # ハイブリッド検索: キーワード検索インデックスとベクトル検索インデックスを構成
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
vector_index.similarity_search("学校が建設される市はどこですか？")

### 4. Graph retriever

#### 4-1. pydanticの補足
グラフ検索用のエンティティを取得する際に、LLMで出力した内容で、Pythonオブジェクトに変換しています。  
ここで簡単に、Pythonのオブジェクトに変換について見ていきます。

In [12]:
from pydantic import BaseModel, Field
from typing import Tuple, List, Optional

In [13]:
class Car(BaseModel):
    name: str = Field(..., title="車名")
    color: str = Field(..., title="色")
    price: int = Field(..., title="価格")

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "テキストから車の名称、色、価格を抽出します。",
        ),
        (
            "human",
            "指定された形式を使用して、以下から情報を抽出します。"
            "input: {question}",
        ),
    ]
)

In [ ]:
# 何もしない場合
chain = prompt|model
chain.invoke("TOYOTAの人気の車は黒のカローラシリーズで、価格は300万円ほどです").content

In [ ]:
# Pythonオブジェクトに変換する場合
chain = prompt|model.with_structured_output(Car)
car = chain.invoke("TOYOTAの人気の車は黒のカローラシリーズで、価格は300万円ほどです")
print(car)
print(car.name, car.color, car.price)

#### 4-2. エンティティの取得
 グラフ検索に使うエンティティを取得

In [17]:

# Extract entities from text
class Entities(BaseModel):
    """エンティティに関する情報の識別"""

    names: List[str] = Field(
        ...,
        description="文章の中に登場する、工事の場所、工事の名前、工事の種類などのエンティティのリスト",
    )

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "テキストから工事の場所、工事の種類、建物の種類のエンティティを抽出します。",
        ),
        (
            "human",
            "指定された形式を使用して、以下から情報を抽出します。"
            "input: {question}",
        ),
    ]
)

In [ ]:
# Pythonオブジェクト変換する場合
entity_chain = prompt | model.with_structured_output(Entities)
entity_chain.invoke({"question": "学校が建設される市はどこですか？"}).names

#### 4-3. graph retriever　実装

In [20]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

# 全文検索インデックスを作成
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# スペルミスを許容する全文検索クエリを生成
def generate_full_text_query(input: str) -> str:
    """
    次の文字列入力に対して「全文検索」クエリを生成します。

    この関数は全文検索に適したクエリ文字列を構築します。
    入力文字列を単語に分割し、各単語に類似性の閾値（最大2文字の変更）を付けて、AND演算子を用いてそれらを結合します。
    ユーザーの質問からデータベースの値にエンティティをマッピングするのに役立ち、多少のスペルミスを許容します。
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [ ]:
"""
 各単語に「2」を付け加えます。これは、Luceneクエリ構文で、最大2文字の変更を許容することを意味します。
 例えば、「apple2」というクエリは「apple」、「aple」、「appla」など、2文字までの違いがある単語もマッチします
"""
generate_full_text_query("病院")

In [22]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    質問に記載されているエンティティの近傍を収集します。
    """
    result = ""
    # エンティティを抽出します
    entities = entity_chain.invoke({"question": question})
    # クエリを実行し、エンティティの近傍を収集します
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, 
            {limit:2})
            YIELD node,score
            CALL {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS 
              output
              UNION
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS 
              output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
print(structured_retriever("学校が建設される市はどこですか？"))

### 5. Structured(Graph)とUnStructured(Vector)を合わせたレトリーバー

In [24]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question, k=10)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:

{"#Document ". join(unstructured_data)}
    """
    return final_data

In [ ]:
retriever("学校が建設される市はどこですか？")

### 6. RAGチェーンの定義

In [29]:
from langchain_core.runnables import (RunnableBranch, RunnableLambda, RunnableParallel,RunnablePassthrough)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

#### 6-1. retrieverの実行


In [ ]:
# 最終的なretrieverの結果
context = retriever("学校が建設される市はどこですか？")
context

In [ ]:
# ベクトル検索のretrieverの結果
unstructured_data = [el.page_content for el in vector_index.similarity_search("学校が建設される市はどこですか？", k=10)]
unstructured_data

#### 6-2. 回答を得る

In [ ]:
template = """
以下のコンテキストに基づいて質問に答えてください。 
{context} 
質問: {question}
 """

prompt = ChatPromptTemplate.from_template(template)
chain = (
    prompt
    | model
    | StrOutputParser()
)

In [ ]:
# Graphとベクトル検索を組み合わせて、質問に答える
chain.invoke({
    "context": context, 
    "question": "学校が建設される市はどこですか？"
    })

In [ ]:
# ベクトル検索のみを使用して、質問に答える
chain.invoke({
    "context": unstructured_data, 
    "question": "学校が建設される市はどこですか？"
    })

#### 6-3. 1つのChainに実装
チャット履歴の対応

目指すコード
```python
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | model
    | StrOutputParser()
)
```

In [33]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

# チャット履歴とその後の質問を要約して、独立した質問に変換する
_template = """
次の「チャット履歴」とそれに「続く質問」をもとに、独立した質問に言い換えてください。
独立した質問は履歴情報が含まれた質問としてください。
チャット履歴:
{chat_history}
続く質問: {question}
"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # 入力にチャット履歴が含まれている場合、それをその後の質問とともに要約します
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | model
        | StrOutputParser(),
    ),
    # それ以外の場合は、チャット履歴がないため、質問のみを通過させます
    RunnableLambda(lambda x : x["question"]),
)

_search_queryの動作確認

In [ ]:
_search_query.invoke({"question": "学校が建設される市はどこですか？"})

In [ ]:
_search_query.invoke(
    {
        "question": "高崎市で行われる工事の概要を教えてください",
        "chat_history": [
            (
                "学校が建設される市はどこですか？",
                "学校が建設される市は以下の通りです：\n\n1. 高崎市\n2. 伊勢崎市\n3. 太田市\n4. 藤岡市\n5. 館林市\n6. 安中市\n\nこれらの市で学校の建設プロジェクトが行われています。"
            )
        ],
    }
)


RunnableParallelで実行する処理の確認

In [36]:
parallel_chain = RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )

In [ ]:
# 質問だけの場合
parallel_chain.invoke({"question": "学校が建設される市はどこですか？"})

In [ ]:
# 会話履歴もある場合
parallel_chain.invoke(
    {
        "question": "高崎市で行われる工事の概要を教えてください",
        "chat_history": [
            (
                "学校が建設される市はどこですか？",
                "学校が建設される市は以下の通りです：\n\n1. 高崎市\n2. 伊勢崎市\n3. 太田市\n4. 藤岡市\n5. 館林市\n6. 安中市\n\nこれらの市で学校の建設プロジェクトが行われています。"
            )
        ],
    }
)

In [39]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "学校が建設される市はどこですか？"})

In [ ]:
chain.invoke(
    {
        "question": "高崎市で行われる工事の概要を教えてください",
        "chat_history": [
            (
                "学校が建設される市はどこですか？",
                "学校が建設される市は以下の通りです：\n1. 高崎市\n2. 伊勢崎市\n3. 太田市\n4. 藤岡市\n5. 館林市\n6. 安中市\n\nこれらの市で学校の建設プロジェクトが行われています。"
            )
        ],
    }
)